In [1]:
import json

In [2]:
all_files = !find ./../data -iname "*.json"|xargs -I{} realpath {}

In [3]:
from collections import defaultdict
tweet_data = defaultdict(set)
for file in all_files:
    with open(file, 'r') as fp:
        for line in fp.readlines():
            cur_data = json.loads(line)
            brand = cur_data['name']
            tweet_data[brand].add(cur_data['tweet'])

In [4]:
for brand, tweet in tweet_data.items():
    print(brand, len(tweet))

Pampers 3220
Gillette 3223
Tide 3220


In [5]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'

model = SentenceTransformer(model_name)
#embeds = model.encode(list(tweet_data['Pampers']))

In [20]:
tweet_embs = dict()
for brand, tweets in tweet_data.items():
    embeds = model.encode(list(tweet_data[brand]), convert_to_numpy=True, batch_size=8)
    tweet_embs[brand] = embeds
    print("done")

done
done
done


In [21]:
import pickle
with open("embeds.pkl", "wb") as fp:
    pickle.dump(tweet_embs, fp)

In [25]:
tweet_embs['Pampers'][0].shape

(768,)

In [46]:
import annoy
from annoy import AnnoyIndex
import random

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for i in range(len(tweet_embs['Tide'])):
    t.add_item(i, tweet_embs['Tide'][i])

t.build(200)

True

In [47]:
t.save('./models/tide.ann')

True

In [49]:
with open("./data/tweets_dump.json", 'w') as fp:
    json.dump({k:list(v) for k, v in tweet_data.items()}, fp)

In [42]:
import re
emb = model.encode("Can my baby advertise for pampers?")
best_3 = t.get_nns_by_vector(emb, 3)
for idx in best_3:
    print(re.sub(r"(?:\@)\S+", "", list(tweet_data['Pampers'])[idx]).strip())
    print("+++++"*10)

Thanks for reaching out to us at Pampers. We're pleased you thought of us. Babies in our ads are selected by the advertising agencies we use. If you would like your child to be in an advertisement, we'd recommend getting in touch with a nearby modeling or talent agency.
++++++++++++++++++++++++++++++++++++++++++++++++++
We're so glad that you'd like your little one to represent Pampers! The babies in our ads are selected by the advertising agencies we use. If you would like your child to be in an advertisement, we'd recommend getting in touch with a nearby modeling or talent agency. Good luck💚
++++++++++++++++++++++++++++++++++++++++++++++++++
How adorable is this! 🤗 We're so happy you want your baby to represent Pampers! The babies in our ads are selected by the advertising agencies we use. If you would like your child to be in an advertisement, we'd recommend getting in touch with a nearby modeling or talent agency.
++++++++++++++++++++++++++++++++++++++++++++++++++
